<H1>How to use NN_CRN

<H5>
Smoothed ReLU : define n1, n2, n3(number of nodes of each layer)/H(smoothing parameter)/init_mu, init_sigma(initialization parameters for the layer)/dt(size of timestep)/k_u(learning rate)/noise/timelen(how many timesteps in one iteration)  

Leaky ReLU : define n1, n2, n3/alpha, beta(Derivative of Leaky ReLU on positive domain and negative domain, respectively)/init_mu, init_sigma, dt, k_u, noise, timelen

Each network has two functions : run, run_for_single_data  

run(label_x, label_y, test_x, test_y, noise_controller, epoch, N, dataset_type) : run for the train dataset(label_x, label_y) and validation dataset(test_x, test_y) for given epoch. N means the size of train dataset. Dataset type is 'XOR' or 'iris' or 'MNIST' or 'half_sine'.(But accuracy data is useless for half_sine dataset) Noise controller is 0 when noise is not used. It is 1 when noise is added on reaction rate. It is 2 when noise is added on each neural network layer. It is 3 when noise is added on input data chemical species. This function returns train loss, validation loss, accuracy, and history of parameters(weight and bias).

run_for_some_data(label_x, label_y, noise_controller, run_num, N) : Execute training for run_num iteration, and returns history of Y value, W1(0,0) value, and derivative of activation function as its input for each layers.

We also provide some visualization tools for each dataset in visualization.py module.

In [ ]:
import NN_CRN
import numpy as np
import datasets
import matplotlib.pyplot as plt
import visualization

In [11]:
Network = NN_CRN.NN_CRN_smoothed_ReLU(n1=2, n2=10, n3=1, H=3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.01, noise=0.1, timelen=300)

In [12]:
label_x, label_y, test_x, test_y, N = datasets.XOR()

In [ ]:
train_loss, validation_loss, accuracy, params = Network.run(label_x = label_x, label_y = label_y, test_x = test_x, test_y = test_y, noise_controller = 0, epoch = 9000, N = N, dataset_type = 'XOR')

Visualization functions

1. XOR_smoothed_ReLU(Network, train_loss, accuracy, params, N, epoch, noise_controller)/XOR_Leaky_ReLU(Network, train_loss, accuracy, params, N, epoch, noise_controller) : returns train loss graph, accuracy graph, training process visualization

2. sine_smoothed_ReLU(Network, train_loss, noise_controller)/sine_Leaky_ReLU(Network, train_loss, noise_controller) : returns train loss graph

3. MNIST_smoothed_ReLU(Network, train_loss, validation_loss, accuracy, N, epoch, noise_controller)/MNIST_Leaky_ReLU(Network, train_loss, validation_loss, accuracy, N, epoch, noise_controller) : returns train loss graph, accuracy graph, classification result

4. iris_smoothed_ReLU(Network, train_loss, validation_loss, accuracy, N, epoch, noise_controller)/iris_Leaky_ReLU(Network, train_loss, validation_loss, accuracy, N, epoch, noise_controller) : returns train loss graph, accuracy graph, classification result

In [ ]:
visualization.XOR_smoothed_ReLU(Network, train_loss, accuracy, params, N=N, epoch=9000, noise_controller=0)

In [5]:
#Code for Figure 4, Figure 6
Yvals, w2vals, Dz1vals, Dz0vals = Network.run_for_some_data(label_x = label_x, label_y = label_y, noise_controller = 0, run_num = 4, N = N)

In [ ]:
plt.plot(Yvals, 'r')
plt.title('Y value')
plt.xlabel('time')
plt.ylabel('Y')
plt.show()

plt.plot(w2vals, 'r')
plt.title('$W^1_{11}$ value')
plt.xlabel('time')
plt.ylabel('$W^1_{11}$')
plt.show()

plt.plot(Dz1vals,'r')
plt.title('$\\frac{d\sigma}{dz^1_0}$ value')
plt.xlabel('time')
plt.ylabel('$\\frac{d\sigma}{dz^1_0}$')
plt.show()

plt.plot(Dz0vals,'r')
plt.title('$\\frac{d\sigma}{dz^0_0}$ value')
plt.xlabel('time')
plt.ylabel('$\\frac{d\sigma}{dz^0_0}$')
plt.show()


<H2>
Example : Mean error about error rate(Figure 7)

In [ ]:
avg_train_loss = [[],[],[],[]]
avg_val_loss = [[],[],[],[]]
for k in range(4) : 
    N1 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.02, timelen=50)
    N2 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.04, timelen=50)
    N3 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.06, timelen=50)
    N4 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.08, timelen=50)
    N5 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.1, timelen=50)
    N6 = NN_CRN.NN_CRN_Leaky_ReLU(n1=1, n2=10, n3=1,alpha=1,beta=0.3, init_mu=0, init_sigma=0.1, dt=0.1, k_u=0.001, noise=0.12, timelen=50)
    N_array = [N1,N2,N3,N4,N5,N6]
    for i in range(6) : 
        train_loss, validation_loss, accuracy, params = N_array[i].run(label_x,label_y,test_x,test_y,noise_controller=1,epoch=100,N=N,dataset_type='sine')
        avg_train_loss[k].append(np.mean([train_loss[(10-j)*(N*10)-1] for j in range(10)]))
        avg_val_loss[k].append(np.mean([validation_loss[(10-j)*(N*10)-1] for j in range(10)]))

plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_train_loss[0],c='r')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_train_loss[1],c='r')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_train_loss[2],c='r')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_train_loss[3],c='r')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_val_loss[0],c='b')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_val_loss[1],c='b')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_val_loss[2],c='b')
plt.scatter([0.02,0.04,0.06,0.08,0.1,0.12],avg_val_loss[3],c='b')
plt.title('Leaky ReLU mean error about noise')
plt.ylim(-0.05,0.2)
plt.xlabel('Noise level')
plt.ylabel('mean error')
plt.legend()
plt.show()
